In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('drive/My Drive/accel'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

import re
%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

plt.rcParams["figure.figsize"] = (10,5)

RANDOM_SEED = 42

In [ ]:
columns=['Benutzer-ID','Aktivitätscode','Zeit','x','y','z']

data_watch_accel = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('drive/My Drive/accel'):
    for filename in filenames:
        data_frame = pd.read_csv('drive/My Drive/accel/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=data_frame.values, columns=columns)
        data_watch_accel=pd.concat([data_watch_accel,temp])

In [ ]:
data_watch_accel['z']=data_watch_accel['z'].str.replace(';','')
data_watch_accel['x']=data_watch_accel['x'].astype('float')
data_watch_accel['y']=data_watch_accel['y'].astype('float')
data_watch_accel['z']=data_watch_accel['z'].astype('float')

data_watch_accel['Aktivitätscode'].value_counts()
data_watch_accel.info()

In [ ]:
data_watch_accel

In [ ]:
data_watch_accel['Aktivitätscode'].value_counts().plot(kind='bar', title='Anzahl der Beschleunigungsmesser-Proben nach Aktivitäten',color=['b','c','g','r','k','y']);

In [ ]:
def plot_activity(activity, data_frame):
    data = data_watch_accel[data_watch_accel['Aktivitätscode'] == activity][['x', 'y', 'z']][:200]
    axis = data["x"].plot(subplots=True, 
                     title=activity,color="b")
    axis = data["y"].plot(subplots=True, 
                 title=activity,color="r")
    axis = data["z"].plot(subplots=True, 
             title=activity,color="g")
    for ax in axis:
        ax.legend(loc='lower left', bbox_to_anchor=(2, 1))

In [ ]:
plot_activity("D", data_frame) #standing

In [ ]:
plot_activity("A", data_frame) #walking

In [ ]:
plot_activity("K", data_frame) #drinking

In [ ]:
plot_activity("Q", data_frame) #writing

In [ ]:
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "B"] #jogging
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "C"] #stairs
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "G"] #brushing teeth
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "M"] #kicking
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "O"] #playingCatch
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "P"] #dribbling
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "S"] #foldingClothes
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "H"] #eatingSoup
data_watch_accel = data_watch_accel[data_watch_accel.Aktivitätscode != "J"] #eatingPasta

In [ ]:
distraction_activity_codes = {'A': 'abgelenkt',
                              'D': 'nicht-abgelenkt',
                              'E': 'nicht-abgelenkt',
                              'F': 'abgelenkt',
                              'I': 'abgelenkt',
                              'K': 'abgelenkt',
                              'L': 'abgelenkt',
                              'Q': 'nicht-abgelenkt',
                              'R': 'abgelenkt'
                              }

In [ ]:
data_watch_accel['Ablenkung'] = data_watch_accel['Aktivitätscode'].map(distraction_activity_codes)

data_watch_accel = data_watch_accel[['Benutzer-ID', 'Ablenkung', 'Aktivitätscode', 'Zeit', 'x', 'y', 'z']]

data_watch_accel

In [ ]:
data_watch_accel['Aktivitätscode'].value_counts()

In [ ]:
activities = data_watch_accel['Aktivitätscode'].value_counts().index

In [ ]:
d_f_a = data_watch_accel[data_watch_accel['Aktivitätscode']=='A'].head(203680)
d_f_k = data_watch_accel[data_watch_accel['Aktivitätscode']=='K'].head(203680)
d_f_e = data_watch_accel[data_watch_accel['Aktivitätscode']=='E'].head(203680)
d_f_d = data_watch_accel[data_watch_accel['Aktivitätscode']=='D'].head(203680)
d_f_l = data_watch_accel[data_watch_accel['Aktivitätscode']=='L'].head(203680)
d_f_f = data_watch_accel[data_watch_accel['Aktivitätscode']=='F'].head(203680)
d_f_q = data_watch_accel[data_watch_accel['Aktivitätscode']=='Q'].head(203680)
d_f_r = data_watch_accel[data_watch_accel['Aktivitätscode']=='R'].head(203680)
d_f_i = data_watch_accel[data_watch_accel['Aktivitätscode']=='I'].head(203680)

In [ ]:
balanced_data = pd.DataFrame()
balanced_data = balanced_data.append([d_f_a,d_f_k,d_f_e,d_f_d,d_f_l,d_f_f,d_f_q,d_f_r,d_f_i]) 

In [ ]:
balanced_data.head()

In [ ]:
label = LabelEncoder()
balanced_data['label'] = label.fit_transform(balanced_data['Ablenkung']) 
balanced_data

In [ ]:
x = balanced_data[['x','y','z','Zeit']]
y = balanced_data['label']

dataset = pd.DataFrame(data=x, columns=['x','y','z','Zeit'])
dataset['label'] = y.values

dataset

In [ ]:
data_watch_accel = dataset.drop(['Zeit'], axis=1)

In [ ]:
time_speed = 20
time_frame_size = time_speed*10 #200
time_hop_size = time_speed*5 #100

def get_frames(df, time_frame_size, time_hop_size):
    
    N_FEATURES = 3
    frames = []
    labels = []
    for i in range(0,len(df )- time_frame_size, time_hop_size):
        x = df['x'].values[i: i+time_frame_size]
        y = df['y'].values[i: i+time_frame_size]
        z = df['z'].values[i: i+time_frame_size]
       
        label = stats.mode(df['label'][i: i+time_frame_size])[0][0]
        frames.append([x,y,z])
        labels.append(label)
        
    frames = np.asarray(frames).reshape(-1, time_frame_size, N_FEATURES)
    labels = np.asarray(labels)
    
    return frames, labels

In [ ]:
x,y = get_frames(dataset, time_frame_size, time_hop_size)

In [ ]:
x.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30, random_state = 0, stratify = y)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

1DCONV MIT MAXPOOLING LAYER

In [ ]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=4, activation = 'relu', input_shape = (200,3)))
model.add(MaxPooling1D(3))

model.add(Conv1D(filters=256, kernel_size=4, activation = 'relu'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

In [ ]:
history = model.fit(x_train, y_train, epochs = 10, verbose=1 )

In [ ]:
score = model.evaluate(x_test, y_test, verbose = 0) 
print('Test Loss: ', score[0]) 
print('Test Accuracy: ', score[1]) 
print(type(model))
y_pred_accel = model.predict_classes(x_test, verbose=1)
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred_accel)
plt.figure(figsize=(7,7))
sns.heatmap(cm, annot=True, fmt="d", cbar=False)
plt.title('Konfusionsmatrix')
plt.ylabel('Tatsächlich')
plt.xlabel('Prognose')

In [ ]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_accel)
print('Precision (tp/(tp+fp)):', precision)
print('Von den', (cm[1][1] + cm[0][1]),'als erfolgreich prognostizierten Projekten wurden', precision * 100,'% also', cm[1][1], 'richtig klassifiziert.')

In [ ]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_pred_accel)
print('Recall (tp/(tp+fn)):', recall)
print('Von den', (cm[1][1] + cm[1][0]),'tatsächlich geförderten Projekten wurden', recall * 100,'% also', cm[1][1], 'richtig klassifiziert.')

In [ ]:
false_positive_rate = (cm[0][1] / (cm[0][1] + cm[0][0]))
print('False Positiv Rate (fpr)):', false_positive_rate)
print('Von den', (cm[0][1] + cm[0][0]),'nicht geförderten Projekten wurden', false_positive_rate * 100,'% also', cm[0][1], 'fälschlicherweise als gefördert klassifiziert.')

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test, y_pred_accel)
print('AUC:', auc)

1D CONV OHNE MAXPOOLING LAYER


In [ ]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=4, activation = 'relu', input_shape = (200,3)))
model.add(Dropout(0.1))

model.add(Conv1D(filters=256, kernel_size=4, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

In [ ]:
history = model.fit(x_train, y_train, epochs = 10, verbose=1 )

In [ ]:
score = model.evaluate(x_test, y_test, verbose = 0) 
print('Test Loss: ', score[0]) 
print('Test Accuracy: ', score[1]) 
print(type(model))
y_pred_accel = model.predict_classes(x_test, verbose=1)
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred_accel)
plt.figure(figsize=(7,7))
sns.heatmap(cm, annot=True, fmt="d", cbar=False)
plt.title('Konfusionsmatrix')
plt.ylabel('Tatsächlich')
plt.xlabel('Prognose')

In [ ]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_accel)
print('Precision (tp/(tp+fp)):', precision)
print('Von den', (cm[1][1] + cm[0][1]),'als erfolgreich prognostizierten Projekten wurden', precision * 100,'% also', cm[1][1], 'richtig klassifiziert.')

In [ ]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_pred_accel)
print('Recall (tp/(tp+fn)):', recall)
print('Von den', (cm[1][1] + cm[1][0]),'tatsächlich geförderten Projekten wurden', recall * 100,'% also', cm[1][1], 'richtig klassifiziert.')

In [ ]:
false_positive_rate = (cm[0][1] / (cm[0][1] + cm[0][0]))
print('False Positiv Rate (fpr)):', false_positive_rate)
print('Von den', (cm[0][1] + cm[0][0]),'nicht geförderten Projekten wurden', false_positive_rate * 100,'% also', cm[0][1], 'fälschlicherweise als gefördert klassifiziert.')

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test, y_pred_accel)
print('AUC:', auc)

CNN MIT CONV2D LAYER

In [ ]:
x_train = x_train.reshape(12831, 200, 3,1)
x_test = x_test.reshape(5499, 200, 3,1)

In [ ]:
model = Sequential()
model.add(Conv2D(128, (2,2), activation = 'relu', input_shape = x_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(256, (2,2), activation = 'relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

In [ ]:
history = model.fit(x_train, y_train, epochs = 10, verbose=1 )

In [ ]:
score = model.evaluate(x_test, y_test, verbose = 0) 
print('Test Loss: ', score[0]) 
print('Test Accuracy: ', score[1]) 
print(type(model))
y_pred_accel = model.predict_classes(x_test, verbose=1)
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred_accel)
plt.figure(figsize=(7,7))
sns.heatmap(cm, annot=True, fmt="d", cbar=False)
plt.title('Konfusionsmatrix')
plt.ylabel('Tatsächlich')
plt.xlabel('Prognose')

In [ ]:
print('Richtig Positiv (True Positive, tp):', cm[1][1])
print('Falsch Positiv (False Positive, fp):', cm[0][1])
print('Falsch Negativ (False Negative, fn):', cm[1][0])
print('Richtig Negativ (True Negative, tn):', cm[0][0])

In [ ]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_accel)
print('Precision (tp/(tp+fp)):', precision)
print('Von den', (cm[1][1] + cm[0][1]),'als erfolgreich prognostizierten Projekten wurden', precision * 100,'% also', cm[1][1], 'richtig klassifiziert.')

In [ ]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_pred_accel)
print('Recall (tp/(tp+fn)):', recall)
print('Von den', (cm[1][1] + cm[1][0]),'tatsächlich geförderten Projekten wurden', recall * 100,'% also', cm[1][1], 'richtig klassifiziert.')

In [ ]:
false_positive_rate = (cm[0][1] / (cm[0][1] + cm[0][0]))
print('False Positiv Rate (fpr)):', false_positive_rate)
print('Von den', (cm[0][1] + cm[0][0]),'nicht geförderten Projekten wurden', false_positive_rate * 100,'% also', cm[0][1], 'fälschlicherweise als gefördert klassifiziert.')

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test, y_pred_accel)
print('AUC:', auc)

In [ ]:
plt.plot(history.history['loss']) 
plt.legend(['training']) 
plt.title('Loss') 
plt.xlabel('epoch')

In [ ]:
plt.plot(history.history['accuracy']) 
plt.legend(['Trainierung']) 
plt.title('Genauigkeit') 
plt.xlabel('Epoche')

RANDOM FOREST FUR TRANSFORMIERTE DATEN

In [ ]:
features = ['ACTIVITY',
            'X0',
            'X1', 
            'X2',
            'X3',
            'X4',
            'X5',
            'X6',
            'X7',
            'X8',
            'X9',
            'Y0',
            'Y1', 
            'Y2',
            'Y3',
            'Y4',
            'Y5',
            'Y6',
            'Y7',
            'Y8',
            'Y9',
            'Z0',
            'Z1', 
            'Z2',
            'Z3',
            'Z4',
            'Z5',
            'Z6',
            'Z7',
            'Z8',
            'Z9',
            'XAVG',
            'YAVG',
            'ZAVG',
            'XPEAK',
            'YPEAK',
            'ZPEAK',
            'XABSOLDEV',
            'YABSOLDEV',
            'ZABSOLDEV',
            'XSTANDDEV',
            'YSTANDDEV',
            'ZSTANDDEV',
            'XVAR',
            'YVAR',
            'ZVAR',
            'XMFCC0',
            'XMFCC1',
            'XMFCC2',
            'XMFCC3',
            'XMFCC4',
            'XMFCC5',
            'XMFCC6',
            'XMFCC7',
            'XMFCC8',
            'XMFCC9',
            'XMFCC10',
            'XMFCC11',
            'XMFCC12',
            'YMFCC0',
            'YMFCC1',
            'YMFCC2',
            'YMFCC3',
            'YMFCC4',
            'YMFCC5',
            'YMFCC6',
            'YMFCC7',
            'YMFCC8',
            'YMFCC9',
            'YMFCC10',
            'YMFCC11',
            'YMFCC12',
            'ZMFCC0', 
            'ZMFCC1',
            'ZMFCC2',
            'ZMFCC3',
            'ZMFCC4',
            'ZMFCC5',
            'ZMFCC6',
            'ZMFCC7',
            'ZMFCC8',
            'ZMFCC9',
            'ZMFCC10',
            'ZMFCC11',
            'ZMFCC12',
            'XYCOS',
            'XZCOS',
            'YZCOS',
            'XYCOR', 
            'XZCOR',
            'YZCOR',
            'RESULTANT', 
            'PARTICIPANT'] 

len(features)

In [ ]:
import glob

path = r'drive/My Drive/accelArff'
all_files = glob.glob(path + "/*.arff")

list_dfs_watch_accel = []

for filename in all_files:

    df = pd.read_csv(filename, names = features, skiprows = 96, index_col=None, header=0)
    list_dfs_watch_accel.append(df)

watch_accel = pd.concat(list_dfs_watch_accel, axis=0, ignore_index=True, sort=False)

watch_accel

In [ ]:
watch_accel = watch_accel[watch_accel.ACTIVITY != "B"] #jogging
watch_accel = watch_accel[watch_accel.ACTIVITY != "C"] #stairs
watch_accel = watch_accel[watch_accel.ACTIVITY != "G"] #brushing teeth
watch_accel = watch_accel[watch_accel.ACTIVITY != "M"] #kicking
watch_accel = watch_accel[watch_accel.ACTIVITY != "O"] #playingCatch
watch_accel = watch_accel[watch_accel.ACTIVITY != "P"] #dribbling
watch_accel = watch_accel[watch_accel.ACTIVITY != "S"] #foldingClothes
watch_accel = watch_accel[watch_accel.ACTIVITY != "H"] #eatingSoup
watch_accel = watch_accel[watch_accel.ACTIVITY != "J"] #eatingPasta

In [ ]:
watch_accel['Ablenkung'] = watch_accel['ACTIVITY'].map(distraction_activity_codes)

watch_accel

In [ ]:
watch_accel['label'] = label.fit_transform(watch_accel['Ablenkung']) 
watch_accel

In [ ]:
watch_accel = watch_accel.drop(['Ablenkung'], axis=1)
watch_accel = watch_accel.drop(['PARTICIPANT'], axis=1)
watch_accel = watch_accel.drop(['ACTIVITY'], axis=1)
watch_accel

In [ ]:
predictors = [x for x in watch_accel.columns if x not in ['label']]
X = watch_accel[predictors]
y = watch_accel.label

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size = 0.75, 
                                                    test_size = 0.25,
                                                    shuffle = True)

In [ ]:
X_train.shape

In [ ]:
y_test.shape

HYPERPARAMETER-OPTIMIERUNG




In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,20,30,40,50],
    'max_features': ['auto','sqrt'],
    'min_samples_leaf': [2,3,4],
    'min_samples_split': [8,9,10],
    'n_estimators': [100, 200, 300, 1000]
}
rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train,y_train)
grid_search.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import time
model_rf = RandomForestClassifier(n_estimators=200,max_features='sqrt',min_samples_leaf=2,min_samples_split=8,
                                 max_depth = 20)
start = time.process_time()
model_rf.fit(X_train, y_train)
print(time.process_time() - start)

In [ ]:
predictions = model_rf.predict(X_test)
predicted_Probability = model_rf.predict_proba(X_test)

In [ ]:
results = X_test.copy()
results['successfull'] = y_test
results['prediction'] = predictions
results['probability'] = predicted_Probability[:,1]

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(results['successfull'], results['prediction'])
print('Accurancy (Klassifikationsgenauigkeit):', accuracy)
print(accuracy * 100, '% der Projekte werden richtig klassifiziert.')

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(results['successfull'], results['prediction'])
plt.figure(figsize=(7,7))
sns.heatmap(cm, annot=True, fmt="d", cbar=False)
plt.title('Konfusionsmatrix')
plt.ylabel('Tatsächlich')
plt.xlabel('Prognose')

In [ ]:
print(f'Model Accuracy: {model_rf.score(X_test,y_test)*100}%')

In [ ]:
print(f'Model Accuracy: {model_rf.score(X_train,y_train)*100}%')

In [ ]:
print('Richtig Positiv (True Positive, tp):', cm[1][1])
print('Falsch Positiv (False Positive, fp):', cm[0][1])
print('Falsch Negativ (False Negative, fn):', cm[1][0])
print('Richtig Negativ (True Negative, tn):', cm[0][0])

In [ ]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, predictions)
print('Precision (tp/(tp+fp)):', precision)
print('Von den', (cm[1][1] + cm[0][1]),'als erfolgreich prognostizierten Projekten wurden', precision * 100,'% also', cm[1][1], 'richtig klassifiziert.')

In [ ]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, predictions)
print('Recall (tp/(tp+fn)):', recall)
print('Von den', (cm[1][1] + cm[1][0]),'tatsächlich geförderten Projekten wurden', recall * 100,'% also', cm[1][1], 'richtig klassifiziert.')

In [ ]:
false_positive_rate = (cm[0][1] / (cm[0][1] + cm[0][0]))
print('False Positiv Rate (fpr)):', false_positive_rate)
print('Von den', (cm[0][1] + cm[0][0]),'nicht geförderten Projekten wurden', false_positive_rate * 100,'% also', cm[0][1], 'fälschlicherweise als gefördert klassifiziert.')

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test, predictions)
print('AUC:', auc)

FEATURE-IMPORTANCE-SCORE

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

plt.rcParams["figure.figsize"] = (5,5)

RANDOM_SEED = 42
# Creating a bar plot
sns.barplot(x=model_rf.feature_importances_, y=X.columns)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Merkmale')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
for name, importance in zip(X.columns, model_rf.feature_importances_):
    print(name, "=", importance)

In [ ]:
feature_imp = pd.Series(model_rf.feature_importances_, index=X.columns)
best_feat = feature_imp.nlargest(10).plot(kind='barh')

In [ ]:
best_features = feature_imp.nlargest(10).index

In [ ]:
x_reduce = X[best_features]

In [ ]:
x_reduce

In [ ]:
from sklearn.model_selection import train_test_split

Xr_train, Xr_test, yr_train, yr_test = train_test_split(x_reduce, y, 
                                                    train_size = 0.75, 
                                                    test_size = 0.25,
                                                    shuffle = True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import time
model_rf = RandomForestClassifier(n_estimators=200,max_features='sqrt',min_samples_leaf=2,min_samples_split=8,
                                 max_depth = 20)
start = time.process_time()
model_rf.fit(Xr_train, yr_train)
print(time.process_time() - start)

In [ ]:
print(f'Model Accuracy: {model_rf.score(Xr_test,yr_test)*100}%')